<a href="https://colab.research.google.com/github/ch-ayesha/RAG-Application-Legal-Case-Summarizer-/blob/main/LEGAL_CASE_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentence-transformers streamlit faiss-cpu datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.169.67.22


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.169.67.22:8501

Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y) y
your url is: https://early-pets-follow.loca.lt
y
2024-10-31 15:53:28.348950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-31 15:53:28.373806: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-31 15:53:28.381242: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-31 15:53:28.398688: I tensorflow/core/platform/cpu_f

In [ ]:
from datasets import load_dataset

# Load the BillSum dataset
dataset = load_dataset("billsum", split="ca_test")  # or "train" or "validation" as needed

# Check the first row of the dataset
first_row = dataset[0]
print(first_row)


{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nThe Legislature finds and declares all of the following:\n(a) (1) Since 1899 congressionally chartered veterans’ organizations have provided a valuable service to our nation’s returning service members. These organizations help preserve the memories and incidents of the great hostilities fought by our nation, and preserve and strengthen comradeship among members.\n(2) These veterans’ organizations also own and manage various properties including lodges, posts, and fraternal halls. These properties act as a safe haven where veterans of all ages and their families can gather together to find camaraderie and fellowship, share stories, and seek support from people who understand their unique experiences. This aids in the healing process for these returning veterans, and ensures their health and happiness.\n(b) As a result of congressional chartering of these veterans’ organizations, the United States Inte

In [ ]:
from datasets import load_dataset

# Load the BillSum dataset (choose the split you want)
dataset = load_dataset("billsum", split="ca_test")  # or "train" or "validation" as needed

# Check the column names of the dataset
column_names = dataset.column_names
print(column_names)


['text', 'summary', 'title']


In [ ]:
# Import Libraries
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
import faiss
import numpy as np
import streamlit as st

# Load the BillSum dataset
dataset = load_dataset("billsum", split="ca_test")

# Initialize models
sbert_model = SentenceTransformer("all-mpnet-base-v2")
t5_tokenizer = AutoTokenizer.from_pretrained("t5-small")
t5_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Prepare data and build FAISS index
texts = dataset["text"][:100]  # Limiting to 100 samples for speed
case_embeddings = sbert_model.encode(texts, convert_to_tensor=True, show_progress_bar=True)
index = faiss.IndexFlatL2(case_embeddings.shape[1])
index.add(np.array(case_embeddings.cpu()))

# Define retrieval and summarization functions
def retrieve_cases(query, top_k=3):
    query_embedding = sbert_model.encode(query, convert_to_tensor=True)
    _, indices = index.search(np.array([query_embedding.cpu()]), top_k)
    return [(texts[i], i) for i in indices[0]]

def summarize_text(text):
    inputs = t5_tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    outputs = t5_model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Streamlit UI
def main():
    st.title("Legal Case Summarizer")
    query = st.text_input("Enter your case search query here:")
    top_k = st.slider("Number of similar cases to retrieve:", 1, 5, 3)

    if st.button("Search"):
        results = retrieve_cases(query, top_k=top_k)
        for i, (case_text, index) in enumerate(results):
            st.subheader(f"Case {i+1}")
            st.write("**Original Text:**", case_text)
            summary = summarize_text(case_text)
            st.write("**Summary:**", summary)

if __name__ == "__main__":
    main()

# Run Streamlit app within Colab
